# ComfyUI停止

In [ ]:
%pip install pandas -q

import subprocess
import pandas as pd
import re

def get_ps_aux():
    """ps aux の出力をDataFrameで取得"""
    result = subprocess.run(["ps", "aux"], stdout=subprocess.PIPE, text=True)
    lines = result.stdout.strip().split("\n")
    header = lines[0].split()
    data = [re.split(r"\s+", line, maxsplit=10) for line in lines[1:]]
    df = pd.DataFrame(data, columns=header)
    return df

def search_processes(df, keywords):
    """特定のキーワードを含むプロセスを抽出"""
    pattern = "|".join(map(re.escape, keywords))
    matched = df[df["COMMAND"].str.contains(pattern, case=False, na=False)]
    return matched.reset_index(drop=True)

def summarize_processes(df):
    """簡易まとめ（ユニークなコマンド一覧）"""
    summary = df["COMMAND"].apply(lambda x: x.split()[0])
    return summary.value_counts().reset_index().rename(columns={"index": "Command", "COMMAND": "Count"})

def find_comfyui_process(df):
    """ComfyUIのプロセスを検出"""
    comfy_keywords = ["comfyui", "main.py", "launch.py", "web.py"]
    return search_processes(df, comfy_keywords)

def can_safely_kill_comfyui(df_comfy):
    """ComfyUIを安全に停止できるかどうか判定"""
    if df_comfy.empty:
        return False, "❌ ComfyUIプロセスが見つかりません。"

    unsafe_parents = ["supervisord", "bash", "sh", "entrypoint.sh"]
    for cmd in df_comfy["COMMAND"]:
        if any(parent in cmd for parent in unsafe_parents):
            return False, "⚠️ ComfyUIは管理プロセス経由で起動されており、停止するとPodごと終了する可能性があります。"

    return True, "✅ ComfyUIは単独プロセスとして起動されており、安全に停止できる可能性が高いです。"

def kill_comfyui(df_comfy):
    """ComfyUIのプロセスを kill する"""
    if df_comfy.empty:
        print("⚠️ 停止対象が存在しません。")
        return

    pids = df_comfy["PID"].tolist()
    print(f"🛑 以下のPIDをkillします: {pids}")

    for pid in pids:
        try:
            subprocess.run(["kill", "-15", pid], check=True)
            print(f"✅ PID {pid} をkillしました。")
        except subprocess.CalledProcessError as e:
            print(f"❌ PID {pid} のkillに失敗しました: {e}")

# ---------- 実行部 ----------

df_all = get_ps_aux()
df_found = search_processes(df_all, ["comfy", "jupyter", "nginx", "supervisord", "code-server", "cloudflared", "ttyd"])
df_comfy = find_comfyui_process(df_all)

print("🔍 関連プロセス一覧:")
display(df_found[["USER", "%CPU", "%MEM", "COMMAND"]])

print("\n🖼️ ComfyUIプロセス詳細:")
if df_comfy.empty:
    print("❌ ComfyUIプロセスは見つかりませんでした。")
else:
    display(df_comfy[["PID", "USER", "%CPU", "%MEM", "COMMAND"]])

# 停止可否の確認
ok, msg = can_safely_kill_comfyui(df_comfy)
print("\n🛑 停止可否の判定:")
print(msg)

# ✅ 安全に停止できると判断された場合だけ kill を確認
if ok and not df_comfy.empty:
    print("\n💬 ComfyUIを停止します...")
    kill_comfyui(df_comfy)
else:
    print("⛔ Podへの影響がある可能性があるため、ComfyUIの停止は行いません。")
